In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import tonic
import numpy as np
#from Network import *
download = False
learn_set = tonic.datasets.NMNIST(save_to='../Data/',
                                  train=True, download=download,
                                  transform=tonic.transforms.AERtoVector()
                                 )

In [3]:
records_path = '../Records'
import datetime
timestr = datetime.datetime.now().date().isoformat()
timestr = '2021-02-13'
verbose = True

%mkdir -p ../Records
%mkdir -p ../Records/EXP_03_NMNIST

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 75
nb_train = int(len(learn_set)//ds)
print(f'The dataset has size {len(learn_set)}, using {nb_train}')

The dataset has size 60000, using 800


In [4]:
import time
def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
# https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html
#criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size
criterion = torch.nn.BCELoss(reduction="mean") # loss divided by output size

class LogisticRegressionModel(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LogisticRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.Softmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [6]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 5 + 1
#num_epochs = 2 ** 9 + 1
# batch_size = 256
n_classes=10
amsgrad = False # gives similar results
amsgrad = True  # gives similar results

def fit_raw_data(dataset, 
            nb_digit,
            learning_rate=learning_rate,
            # batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, #**kwargs
        ):
    

    generator = torch.Generator().manual_seed(42)
    sampler = torch.utils.data.RandomSampler(dataset, replacement=True, num_samples=nb_digit, generator=generator)
    loader = tonic.datasets.DataLoader(dataset, sampler=sampler)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')

    N = 34*34*2
    n_classes = 10
    logistic_model = LogisticRegressionModel(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    
    for epoch in range(int(num_epochs)):
        losses = []
        for X, label in loader:
            X, label = X.to(device), label.to(device)
            X, label = X.squeeze(0), label.squeeze(0) # just one digit = one batch
            outputs = logistic_model(X)

            n_events = X.shape[0]
            #print(X.squeeze(0).shape, label * torch.ones((1, n_events)))
            #print(outputs, label)
            labels = label*torch.ones(n_events).type(torch.LongTensor).to(device)
            labels = torch.nn.functional.one_hot(labels, num_classes=n_classes).type(torch.DoubleTensor).to(device)
            #print(outputs.shape, labels.shape)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")
            
    return logistic_model, losses

In [7]:
tic()
model, loss = fit_raw_data(learn_set, 
            nb_train,
            learning_rate=learning_rate,
            # batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=True,
        )
toc()

device -> cpu
Iteration: 0 - Loss: 0.30797
Iteration: 1 - Loss: 0.27139
Iteration: 2 - Loss: 0.24503
Iteration: 3 - Loss: 0.25235
Iteration: 4 - Loss: 0.27811
Iteration: 5 - Loss: 0.30568
Iteration: 6 - Loss: 0.22946
Iteration: 7 - Loss: 0.25978
Iteration: 8 - Loss: 0.25760
Iteration: 9 - Loss: 0.23459
Iteration: 10 - Loss: 0.29423
Iteration: 11 - Loss: 0.28655
Iteration: 12 - Loss: 0.24020
Iteration: 13 - Loss: 0.27124
Iteration: 14 - Loss: 0.26455
Iteration: 15 - Loss: 0.22961
Iteration: 16 - Loss: 0.22876
Iteration: 17 - Loss: 0.23239
Iteration: 18 - Loss: 0.25138
Iteration: 19 - Loss: 0.21375
Iteration: 20 - Loss: 0.21686
Iteration: 21 - Loss: 0.25900
Iteration: 22 - Loss: 0.20008
Iteration: 23 - Loss: 0.23550
Iteration: 24 - Loss: 0.21693
Iteration: 25 - Loss: 0.30183
Iteration: 26 - Loss: 0.25443
Iteration: 27 - Loss: 0.25170
Iteration: 28 - Loss: 0.23566
Iteration: 29 - Loss: 0.25216
Iteration: 30 - Loss: 0.23054
Iteration: 31 - Loss: 0.21808
Iteration: 32 - Loss: 0.25373
Done i

In [8]:
test_set = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False, download=download,
                                transform=tonic.transforms.AERtoVector())

In [9]:
nb_test = int(len(test_set)//ds)
print(f'The dataset has size {len(test_set)}, using {nb_test}')

The dataset has size 10000, using 133


In [10]:
def predict_data(test_set, model, # gamma=gamma,
            verbose=False, **kwargs
        ):
    
    with torch.no_grad():

        generator=torch.Generator().manual_seed(42)
        sampler = torch.utils.data.RandomSampler(test_set, replacement=True, num_samples=nb_test, generator=generator)
        loader = tonic.datasets.DataLoader(test_set, sampler=sampler)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        logistic_model = model.to(device)

        pred_target, true_target = [], []

        for X, label in loader:
            X = X.to(device)
            X, label = X.squeeze(0), label.squeeze(0)

            n_events = X.shape[0]
            labels = label*torch.ones(n_events).type(torch.LongTensor)

            outputs = logistic_model(X)

            pred_target.append(torch.argmax(outputs, dim=1).cpu().numpy())
            true_target.append(labels.numpy())

    return pred_target, true_target

In [11]:
pred_target, true_target = predict_data(test_set, model)

In [12]:
pred_target, true_target

([array([1, 1, 7, ..., 7, 7, 7]),
  array([1, 1, 1, ..., 6, 6, 6]),
  array([1, 1, 1, ..., 6, 6, 6]),
  array([1, 1, 1, ..., 6, 6, 6]),
  array([7, 1, 3, ..., 0, 0, 0]),
  array([1, 1, 1, ..., 7, 7, 7]),
  array([7, 7, 7, ..., 8, 8, 8]),
  array([7, 1, 1, ..., 1, 1, 1]),
  array([4, 7, 7, ..., 8, 8, 8]),
  array([1, 7, 7, ..., 7, 7, 7]),
  array([1, 1, 1, ..., 1, 1, 1]),
  array([7, 7, 7, ..., 9, 9, 9]),
  array([1, 7, 7, ..., 7, 7, 7]),
  array([1, 7, 7, ..., 3, 3, 3]),
  array([1, 1, 7, ..., 8, 8, 8]),
  array([1, 3, 7, ..., 7, 7, 7]),
  array([7, 1, 1, ..., 7, 7, 7]),
  array([7, 7, 1, ..., 8, 8, 8]),
  array([1, 1, 1, ..., 8, 8, 8]),
  array([1, 1, 1, ..., 9, 9, 9]),
  array([5, 1, 1, ..., 9, 9, 9]),
  array([1, 7, 7, ..., 8, 8, 8]),
  array([1, 1, 1, ..., 1, 1, 1]),
  array([1, 1, 1, ..., 1, 1, 1]),
  array([1, 1, 1, ..., 3, 3, 3]),
  array([7, 1, 1, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 9, 9, 9]),
  array([7, 7, 7, ..., 4, 4, 4]),
  array([1, 1, 1, ..., 2, 2, 2]),
  array([1, 1,

In [13]:
for pred_target_, true_target_ in zip(pred_target, true_target):
    print(len(pred_target_), len(true_target_))
    print(len(pred_target_ == true_target_))
    print(np.mean((pred_target_ == true_target_)*1.))

3808 3808
3808
0.9994747899159664
4645 4645
4645
0.9745963401506996
3595 3595
3595
0.9874826147426982
4304 4304
4304
0.9769981412639405
5272 5272
5272
0.9850151745068285
4337 4337
4337
0.8033202674659903
4947 4947
4947
0.9777643015969274
1598 1598
1598
0.9993742177722152
3659 3659
3659
0.12462421426619295
3479 3479
3479
0.997413049726933
2483 2483
2483
1.0
4235 4235
4235
0.9943329397874853
3379 3379
3379
0.9997040544539805
4791 4791
4791
0.9169275725318305
5527 5527
5527
0.988420481273747
4186 4186
4186
0.9995222169135213
4492 4492
4492
0.9508014247551202
4006 4006
4006
0.9952571143285073
5346 5346
5346
0.98989898989899
3102 3102
3102
0.8175370728562218
4481 4481
4481
0.9064940861414863
4491 4491
4491
0.15185927410376307
2734 2734
2734
0.9842721287490855
2880 2880
2880
1.0
6213 6213
6213
0.9233864477708031
4953 4953
4953
0.9917221885725823
3396 3396
3396
0.9151943462897526
3452 3452
3452
0.9979721900347625
4508 4508
4508
0.9944543034605147
4508 4508
4508
0.9966725820763088
5550 5550
55

In [14]:
accuracy = []
for pred_target_, true_target_ in zip(pred_target, true_target):
    accuracy.append(np.mean(pred_target_ == true_target_))
print(f'{np.mean(accuracy)=}')    

np.mean(accuracy)=0.8046414454320989
